### Bonus: Scraping in order to fill our ethnicity_labels column

In [2]:
%matplotlib inline
import pandas as pd
import requests
import numpy as np
import requests
from bs4 import BeautifulSoup
import random
import time

In [3]:
# Loading our cleaned actor dataset:

actor = pd.read_csv('./cleaned_datasets/actor.csv', names=['actor_name', 'free_actor_id', 'actor_birth', 'actor_gender', 'actor_ethnicity', 'actor_height', 'actor_ethnicity_label'])

/var/folders/zv/w1j6q4wj18n431c9sx_sxczh0000gn/T/ipykernel_69218/2964010220.py:3: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  actor = pd.read_csv('./cleaned_datasets/actor.csv', names=['actor_name', 'free_actor_id', 'actor_birth', 'actor_gender', 'actor_ethnicity', 'actor_height', 'actor_ethnicity_label'])


In [4]:
actor['actor_ethnicity'].isna().sum()

127610

In [5]:
actor.head(50)

,actor_name,free_actor_id,actor_birth,actor_gender,actor_ethnicity,actor_height,actor_ethnicity_label
0,actor_name,free_actor_id,actor_birth,actor_gender,actor_ethnicity,actor_height,actor_ethnicity_label
1,Rosa Maria Sardà,/m/0gh6sw,1941.0,F,/m/03ttfc,NaN,Spaniards
2,Mercè Pons,/m/0267qhz,1966.0,F,NaN,NaN,Unknown
3,Anna Lizaran,/m/0263499,1944.0,F,NaN,NaN,Unknown
4,Núria Espert,/m/0263yvy,1935.0,F,NaN,NaN,Unknown
5,Ted Raimi,/m/07qn0,1965.0,M,NaN,1.87,Unknown
6,Ivan Raimi,/m/0gmjf8,1956.0,M,NaN,NaN,Unknown
7,Bruce Campbell,/m/01h8f,1958.0,M,NaN,1.85,Unknown
8,Bridget Fonda,/m/01yd8v,1964.0,F,/m/07hwkr,1.68,White Americans
9,Embeth Davidtz,/m/034b7r,1965.0,F,/m/06v41q,1.73,French Americans


#### Scrap with  wikidata

In [6]:
# Lists of fake headers for web scraping
user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
]

# Function to scrape Wikidata for country of citizenship information
def scrape_wikidata_citizenship(actor_name):
    search_url = f"https://www.wikidata.org/w/index.php?search={actor_name.replace(' ', '+')}&title=Special%3ASearch&profile=advanced&fulltext=1&ns0=1&ns120=1"

    try:
        # Request the Wikidata search results page
        response = requests.get(search_url, headers={'User-Agent': random.choice(user_agents_list)})
        
        # If the response is not successful
        if response.status_code != 200:
            print(f"Failed to fetch the webpage for {actor_name}. Status Code: {response.status_code}")
            return None

        # Parse the HTML for the search results
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the first search result link
        search_results = soup.select('li.mw-search-result a')
        if not search_results:
            print(f"No matching actor found for {actor_name} in Wikidata.")
            return None

        # Navigate to the actor's page (assuming the first result is correct)
        actor_page_url = f"https://www.wikidata.org{search_results[0]['href']}"
        actor_response = requests.get(actor_page_url, headers={'User-Agent': random.choice(user_agents_list)})
        
        # If the response is not successful
        if actor_response.status_code != 200:
            print(f"Failed to fetch the actor page for {actor_name}. Status Code: {actor_response.status_code}")
            return None

        # Parse the actor page HTML
        actor_soup = BeautifulSoup(actor_response.content, 'html.parser')
        
        # Look for country of citizenship
        citizenship_div = actor_soup.find_all('div', class_='wikibase-statementgroupview')
        for div in citizenship_div:
            label = div.find('div', class_='wikibase-statementgroupview-property-label')
            if label and 'country of citizenship' in label.text.lower():
                # Extract the country of citizenship
                country_tag = div.find('div', class_='wikibase-snakview-value')
                if country_tag:
                    country = country_tag.text.strip()
                    print(f"Actor Name: {actor_name}, Country of Citizenship: {country}")
                    return country

        print(f"Country of citizenship not found for {actor_name}.")
        return None

    except Exception as e:
        print(f"Error while trying to access {actor_name}'s webpage: {e}")
        return None

# Example DataFrame to work with
actor_df = pd.DataFrame({'actor_name': ['Johnny Depp', 'Leonardo DiCaprio', 'Samuel L. Jackson']})

# Iterate through all actor names and scrape the country of citizenship information
citizenship_data = []

for _, row in actor_df.iterrows():
    actor_name = row['actor_name']
    country = scrape_wikidata_citizenship(actor_name)
    
    # Add the data to our list
    citizenship_data.append({'actor_name': actor_name, 'country_of_citizenship': country})
    
    # Adding a small delay to prevent overwhelming the server
    time.sleep(random.uniform(1, 3))  # Random delay between 1 and 3 seconds

# Print out the results
print(citizenship_data)


Actor Name: Johnny Depp, Country of Citizenship: United States of America
Actor Name: Leonardo DiCaprio, Country of Citizenship: United States of America
Actor Name: Samuel L. Jackson, Country of Citizenship: United States of America
[{'actor_name': 'Johnny Depp', 'country_of_citizenship': 'United States of America'}, {'actor_name': 'Leonardo DiCaprio', 'country_of_citizenship': 'United States of America'}, {'actor_name': 'Samuel L. Jackson', 'country_of_citizenship': 'United States of America'}]


In [ ]:
# Assume actor.csv is the dataset and it contains 'actor_name' and 'actor_ethnicity_label' columns
actor_df = pd.read_csv('cleaned_datasets/actor.csv')

# Iterate through all actor names and scrape the country of citizenship information for unknown values
for index, row in actor_df.iterrows():
    if row['actor_ethnicity_label'] == 'Unknown' and isinstance(row['actor_name'], str):
        actor_name = row['actor_name']
        country = scrape_wikidata_citizenship(actor_name)

        # Update the DataFrame if a country is found
        if country:
            actor_df.at[index, 'actor_ethnicity_label'] = country

        # Adding a small delay to prevent overwhelming the server
        time.sleep(random.uniform(1, 3))  # Random delay between 1 and 3 seconds

# Save the updated DataFrame
actor_df.to_csv('updated_actor.csv', index=False)

print("Actor dataset updated successfully.")

#### Scrap with the website ethnicelebs

In [7]:
import requests
import random

# Lists of fake headers 
user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 10; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Mobile Safari/537.36',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Version/14.0 Mobile/15A372 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 OPR/77.0.4054.203',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
    'Mozilla/5.0 (Linux; U; Android 8.1.0; en-US; Pixel 2 XL Build/OPM2.171019.029) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/70.0.3538.110 Mobile Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 9; SAMSUNG SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/10.1 Chrome/71.0.3578.99 Mobile Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0',
    'Mozilla/5.0 (Linux; U; Android 9; en-US; Redmi Note 7 Pro Build/PKQ1.181203.001) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/74.0.3729.157 Mobile Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'
]

# Function to test accessing the webpage
def test_webpage_access(actor_name):
    formatted_name = actor_name.lower().replace(' ', '-')
    search_url = f"https://ethnicelebs.com/{formatted_name}"

    try:
        response = requests.get(search_url, headers={'User-Agent': random.choice(user_agents_list)})
        if response.status_code == 200:
            print(f"Successfully accessed the webpage for {actor_name}")
        else:
            print(f"Failed to access the webpage for {actor_name}, Status Code: {response.status_code}")
    except Exception as e:
        print(f"Error while trying to access {actor_name}'s webpage: {e}")

# Test with an example actor name
actor_name = 'Scarlett Johansson'
test_webpage_access(actor_name)


Successfully accessed the webpage for Scarlett Johansson


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Lists of fake headers 
user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 10; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Mobile Safari/537.36',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Version/14.0 Mobile/15A372 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 OPR/77.0.4054.203',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
    'Mozilla/5.0 (Linux; U; Android 8.1.0; en-US; Pixel 2 XL Build/OPM2.171019.029) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/70.0.3538.110 Mobile Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36',
    'Mozilla/5.0 (Linux; Android 9; SAMSUNG SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/10.1 Chrome/71.0.3578.99 Mobile Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0',
    'Mozilla/5.0 (Linux; U; Android 9; en-US; Redmi Note 7 Pro Build/PKQ1.181203.001) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/74.0.3729.157 Mobile Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'
]

ethnicities = {}

# Function to get ethnicity from ethnicelebs.com
def get_ethnicity_from_ethnicelebs(actor_name):
    formatted_name = actor_name.lower().replace(' ', '-')
    search_url = f"https://ethnicelebs.com/{formatted_name}"

    try:
        response = requests.get(search_url, headers={'User-Agent': random.choice(user_agents_list)})
        if response.status_code != 200:
            print(f"Page not found for {formatted_name}")
            return 'Unknown'

        print(f"Page found for {formatted_name}")
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Print the formatted HTML in order to then ctrlF the data interesting to us and  
        # infer the html markers to feed to soup.find, this yields for RDJ:
        #print(soup.prettify())
        """<p>
             <strong>
                  Ethnicity: Ashkenazi Jewish (three eighths), German, Swiss-German, Irish, English, Scottish, remote Austrian
             </strong>
            </p>"""


        # Find the ethnicity section
        ethnicity_section = soup.find('p', text=lambda t: t and 'Ethnicity:' in t)
        if ethnicity_section:
            ethnicity = ethnicity_section.find_next('strong').text
            return ethnicity
            

        return 'Unknown'  # Default if no specific section is found

    except Exception as e:
        print(f"Error fetching data for {actor_name}: {e}")
        return 'Unknown'

# List of actors to check
actor_names = ['Robert Downey Jr.']
actor_names_extended = ['Robert Downey Jr.', 'Scarlett Johansson', 'Chris Evans']
all_actors = actor[actor['actor_ethnicity_label'] == 'Unknown']['actor_name']

# Populate the DataFrame
count = 0
for actor_name in all_actors:
    ethnicity = get_ethnicity_from_ethnicelebs(actor_name)
    ethnicities[actor_name] = ethnicity
    if(count%50 == 0):
        time.sleep(1)
    count += 1

# Print the resulting DataFrame
print(ethnicities)
